# Bell with clapper

In this section, we also model the clapper moving in response to the bell.

In [4]:
%matplotlib inline
import scipy
import scipy.integrate as integrate
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
from functools import partial
from ipywidgets import interactive, FloatSlider, Layout
from IPython.display import display, SVG, HTML, Image, clear_output
import svgutils
import copy
import time
#from bqplot import pyplot as plt


Some constants. To see what bellringing on the moon would be like, set the acceleration
due to gravity to 1.62.  Or 24.5 for Jupiter.

The clapper bounds are the angles where the clapper strikes the bell. We've set these
to +/- 30 degrees here somewhat arbitarily.

In [5]:
# acceleration due to gravity
g = 9.81

# clapper bounds
phi0 = -30 * np.pi / 180
phi1 = 30 * np.pi / 180

In [7]:
# initial conditions

# angle of the bell
theta_initial = 178

# now we have 4 variables for the position and velocity of the
# bell and the clapper: theta, theta_dot, phi, phi_dot
y0, t0 = [theta_initial * np.pi / 180, 0.0, phi1, 0.0], 0.0

The equations of motion are specified by the derivatives of the variables.
In this case, we include the clapper which moves in response to the bell,
but we *do not* yet include the back-action of the clapper on the bell.

In [8]:
def deriv(l_b, l_c, c, alpha, y, t):
    [theta, vtheta, phi, vphi] = y
    theta_dot  = vtheta
    vtheta_dot = - (g / l_b) * np.sin(theta)

    phi_dot = vphi

    # note sign error in the original notes 
    vphi_dot = -vtheta_dot * (1 + (c / l_c) * np.cos(phi - alpha)) \
               + (c / l_c) * (vtheta**2) * np.sin(phi - alpha) - (g / l_c) * np.sin(theta+phi)
    return np.array([theta_dot, vtheta_dot, phi_dot, vphi_dot])

Because we need to clip the clapper position at the side of the bell, we cannot use the NumPy
built-in ODE solver. So we use here a custom 4th order Runge-Kutta. Note the rk4() function is completely generic - the integrate_rk4 function takes the clapper into account

In [24]:
def rk4(f, x, t, step):
    k1 = step * f(x, t)
    x1 = x + 0.5*k1
    k2 = step * f(x1, t+0.5*step)
    x2 = x + 0.5*k2
    k3 = step * f(x2, t+0.5*step)
    x3 = x + k3;
    k4 = step * f(x3, t+step)
    return (k1 + 2*k2 + 2*k3 + k4) / 6.0

In [60]:
def integrate_rk4_clapper(y0, t, step, l_b, l_c, c, alpha, clapper):
    y = y0 + rk4(partial(deriv, l_b, l_c, c, alpha), y0, t, step)
    # check for clapper clapping the bell
    if y[2] > phi1:
        clapper = True
        y[2] = phi1
        y[3] = 0
    elif y[2] < phi0:
        clapper = True
        y[2] = phi0
        y[3] = 0
    else:
        clapper = False
    return [clapper, y]

In [61]:
def integrate_sequence(y0, tindex, l_b, l_c, c, alpha):
    y = []
    StrikeTimes = []
    y.append(y0)
    t_last = tindex[0]
    clapper = True
    for t in tindex[1:]:
        step = t - t_last
        OldClapper = clapper
        [clapper, yNext] = integrate_rk4_clapper(y[-1], t_last, step, l_b, l_c, c, alpha, clapper)
        y.append(yNext)
        if clapper and not OldClapper:
            StrikeTimes.append(t)
        t_last = t
    return [StrikeTimes, np.array(y)]

In [62]:
# period of small oscillations
T0 = 2 * np.pi * np.sqrt(l_b / g)
print('Period of small oscillations: {:.4} seconds'.format(T0))

Period of small oscillations: 1.554 seconds


In [88]:
# Exact formula from elliptic integral for the period
Texact = 4 * np.sqrt(l_b / g) * scipy.special.ellipk(np.sin(y0[0]/2)**2)
print('Exact period: {:.4} seconds'.format(Texact))

Exact period: 5.376 seconds


In [91]:
def update(l_b, l_c, c, alpha):
    Texact = 4 * np.sqrt(l_b / g) * scipy.special.ellipk(np.sin(y0[0]/2)**2)
    t = np.linspace(t0, 2*Texact, 10000)
    [StrikeTimes, y] = integrate_sequence(np.array(y0), t, l_b, l_c, c, alpha*np.pi/180)
    plt.figure(figsize=(10,8))
    plt.plot(t, y[:,0] * 180 / np.pi, 'b', label='bell')
    plt.plot(t, y[:,2] * 180 / np.pi, 'g', label='clapper')
    plt.plot([], [], 'r-', label='strike')
    for tStrike in StrikeTimes:
        plt.plot((tStrike, tStrike), (-180, 180), 'r-')
    plt.legend(loc='best')
    plt.xlabel('t')
    plt.ylabel('angle')
    plt.grid()
    plt.show()
    print('Period: {:.4}'.format(Texact))
    print('Oddstruckness: {:.4}'.format(2*StrikeTimes[1]-StrikeTimes[0] - StrikeTimes[2]))

In [92]:
mylayout = Layout(width='50%', height='20px')
l_b_float = FloatSlider(description=r'\(l_b\)', \
                        min=0.01, max=5, step=0.01, value=0.6, \
                        continuous_update=False, layout=mylayout)
l_c_float = FloatSlider(description=r'\(l_c\)', \
                        min=0.01, max=5, step=0.01, value=1, \
                        continuous_update=False, layout=mylayout)
c_float = FloatSlider(description=r'\(c\)', \
                      min=0.01, max=5, step=0.01, value=0.2, \
                      continuous_update=False, layout=mylayout)
alpha_float = FloatSlider(description=r'\(\alpha\)', \
                          min=-45, max=45, step=0.1, value=0, \
                          continuous_update=False, layout=mylayout)
interactive(update, l_b = l_b_float, l_c = l_c_float, c = c_float, alpha=alpha_float)

interactive(children=(FloatSlider(value=0.6, continuous_update=False, description='\\(l_b\\)', layout=Layout(height='20px', width='50%'), max=5.0, min=0.01, step=0.01), FloatSlider(value=1.0, continuous_update=False, description='\\(l_c\\)', layout=Layout(height='20px', width='50%'), max=5.0, min=0.01, step=0.01), FloatSlider(value=0.2, continuous_update=False, description='\\(c\\)', layout=Layout(height='20px', width='50%'), max=5.0, min=0.01, step=0.01), FloatSlider(value=0.0, continuous_update=False, description='\\(\\alpha\\)', layout=Layout(height='20px', width='50%'), max=45.0, min=-45.0), Output()), _dom_classes=('widget-interact',))

In [18]:
svg = svgutils.transform.fromfile('wheel_bell.svg')
image_width = float(svg.width)
image_height = float(svg.height)
bell_image = svg.getroot()

def get_bell_image(angle):
    bell_pic = copy.deepcopy(bell_image)
    bell_pic.rotate(-angle, image_width/2, image_height/2)
    return bell_pic

def show_bell(angle):
    bellfig = svgutils.transform.SVGFigure(image_width, image_height)
    bellfig.append(get_bell_image(angle))
    display(SVG(bellfig.to_str()))

In [20]:
interactive(show_bell, angle = FloatSlider(min=-180,max=180,step=1))

interactive(children=(FloatSlider(value=0.0, description='angle', max=180.0, min=-180.0, step=1.0), Output()), _dom_classes=('widget-interact',))

In [95]:
angle = -180
while True:
    clear_output(wait=True)
    show_bell(angle)
    angle = angle + 1
    time.sleep(0.03)

KeyboardInterrupt: 